In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
from load_data import merged_df

In [ ]:
merged_df()

In [5]:
#helper function to convert datetime strings to integers representing a time year-month-day hour-min-sec
from datetime import datetime
def datestr_to_int(datetime_str,date_format):
    return datetime.strptime(datetime_str, date_format).timestamp()

In [71]:
#load the training data, dropping NaN's
train = pd.read_csv("train.csv").dropna()
train['datetime'] = train['datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))

In [72]:
gas_prices = pd.read_csv("gas_prices.csv")
#convert date strings to ints
gas_prices['forecast_date'] = gas_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d'))
gas_prices = gas_prices.drop(columns=['origin_date'])
gas_prices['data_block_id'] -= 1

In [73]:
electricity_prices = pd.read_csv("electricity_prices.csv")
#convert date strings to ints
electricity_prices['forecast_date'] = electricity_prices['forecast_date'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S'))
electricity_prices = electricity_prices.drop(columns=['origin_date'])
electricity_prices['data_block_id'] -= 1

In [74]:
forecast_weather = pd.read_csv("forecast_weather.csv")
#convert strings to ints
forecast_weather = forecast_weather.drop(columns=['origin_datetime'])
forecast_weather['forecast_datetime'] = forecast_weather['forecast_datetime'].apply(lambda x: datestr_to_int(x,'%Y-%m-%d %H:%M:%S%z'))
forecast_weather = forecast_weather.rename(columns={'forecast_datetime':'forecast_date','origin_datetime':'origin_date'})
forecast_weather['data_block_id'] -= 1

In [11]:
#https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455100

In [75]:
#merge gas prices and train.csv data
#column names differ, so use left_on and right_on
df = pd.merge(train, gas_prices, left_on=['data_block_id','datetime'], right_on=['data_block_id','forecast_date'], how='left')

In [76]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,highest_price_per_mwh
0,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,45.23,46.32
1,0,0,1,96.590,1,1.630469e+09,0,1,0,1.630469e+09,45.23,46.32
2,0,0,2,0.000,0,1.630469e+09,0,2,1,1.630469e+09,45.23,46.32
3,0,0,2,17.314,1,1.630469e+09,0,3,1,1.630469e+09,45.23,46.32
4,0,0,3,2.904,0,1.630469e+09,0,4,2,1.630469e+09,45.23,46.32
...,...,...,...,...,...,...,...,...,...,...,...,...
2011709,15,1,0,181.786,1,1.685419e+09,636,2012237,64,1.685419e+09,29.00,34.00
2011710,15,1,1,0.000,0,1.685419e+09,636,2012238,59,1.685419e+09,29.00,34.00
2011711,15,1,1,28.832,1,1.685419e+09,636,2012239,59,1.685419e+09,29.00,34.00
2011712,15,1,3,0.000,0,1.685419e+09,636,2012240,60,1.685419e+09,29.00,34.00


In [77]:
#merge train and gas_prices via left join on data_block_id
#this leaves all rows of train, but matches
df = df.merge(electricity_prices, on=['data_block_id','forecast_date'], how='left')

In [78]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh
0,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,45.23,46.32,92.51
1,0,0,1,96.590,1,1.630469e+09,0,1,0,1.630469e+09,45.23,46.32,92.51
2,0,0,2,0.000,0,1.630469e+09,0,2,1,1.630469e+09,45.23,46.32,92.51
3,0,0,2,17.314,1,1.630469e+09,0,3,1,1.630469e+09,45.23,46.32,92.51
4,0,0,3,2.904,0,1.630469e+09,0,4,2,1.630469e+09,45.23,46.32,92.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011709,15,1,0,181.786,1,1.685419e+09,636,2012237,64,1.685419e+09,29.00,34.00,8.57
2011710,15,1,1,0.000,0,1.685419e+09,636,2012238,59,1.685419e+09,29.00,34.00,8.57
2011711,15,1,1,28.832,1,1.685419e+09,636,2012239,59,1.685419e+09,29.00,34.00,8.57
2011712,15,1,3,0.000,0,1.685419e+09,636,2012240,60,1.685419e+09,29.00,34.00,8.57


In [80]:
#merge forecast weather data on data_block_id and forecast_date via left join
df = df.merge(forecast_weather, on=['data_block_id','forecast_date'],how='left')

In [81]:
df.dropna()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,...,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,...,0.981354,0.206161,0.000000,0.985138,0.718047,-9.847090,33.148941,42.714209,0.0,0.000000e+00
1,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,...,0.973724,0.029022,0.000000,0.974396,1.053862,-5.797529,46.215608,47.822542,0.0,0.000000e+00
2,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,...,0.977783,0.045532,0.000061,0.978775,2.441680,-6.653486,52.211163,53.488653,0.0,5.364418e-07
3,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,...,0.949554,0.040604,0.000000,0.951691,2.822661,-8.592695,77.846719,60.980320,0.0,0.000000e+00
4,0,0,1,0.713,0,1.630469e+09,0,0,0,1.630469e+09,...,0.868027,0.065216,0.020096,0.876770,2.863921,-8.824141,137.593385,69.199765,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11332157,15,1,3,170.135,1,1.685419e+09,636,2012241,60,1.685419e+09,...,0.000000,0.002075,0.120850,0.122345,3.244002,1.453518,645.835293,303.716111,0.0,0.000000e+00
11332158,15,1,3,170.135,1,1.685419e+09,636,2012241,60,1.685419e+09,...,0.000000,0.000000,0.365601,0.365601,4.402694,1.083645,608.875293,300.778333,0.0,0.000000e+00
11332159,15,1,3,170.135,1,1.685419e+09,636,2012241,60,1.685419e+09,...,0.000000,0.000000,0.562164,0.562164,3.980086,-0.196629,412.341960,265.933889,0.0,0.000000e+00
11332160,15,1,3,170.135,1,1.685419e+09,636,2012241,60,1.685419e+09,...,0.000000,0.000000,0.293518,0.293518,4.573836,-2.191746,107.168626,179.449444,0.0,0.000000e+00


In [61]:
#drop NaN rows
df = df.dropna()

In [22]:
#drop the target column and data_block_id as they are not present in test data
X = df.drop(['target'], axis=1).rename(columns={'datetime': 'prediction_datetime'})

In [23]:
X.columns

Index(['county', 'is_business', 'product_type', 'is_consumption',
       'prediction_datetime', 'data_block_id', 'row_id', 'prediction_unit_id',
       'forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'latitude', 'longitude', 'hours_ahead', 'temperature',
       'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid',
       'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component', 'direct_solar_radiation',
       'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation'],
      dtype='object')

In [24]:
#set the target variable
y = df['target']

In [25]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [26]:
# load the modeling modules from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [27]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
#create a multivariable linear regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.feature_names = list(X_train.columns.values)

In [29]:
#predict on the test data
lin_pred = regr.predict(X_test)

In [30]:
mean_absolute_error(lin_pred,y_test)

287.75679632595796

In [31]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)
rf.feature_names = list(X_train.columns.values)

In [32]:
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)

In [33]:
#mean absolute error for random regressor
mean_absolute_error(y_test,rf_pred)

0.1559591809676255

In [34]:
#perform k-fold cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #train the model
    lin_regr = linear_model.LinearRegression()
    lin_regr.fit(X_train, y_train)
    #predict on the test data
    lin_pred = regr.predict(X_test)
    #evaluate the model
    print(mean_absolute_error(lin_pred,y_test))

273.4827216079995
285.00468175480364
269.84207813612034
297.5972492682266
309.99922880975254


In [36]:
import pickle
# save the linear model to disk
linear_filename = '../models/linear_model.sav'
pickle.dump(regr, open(linear_filename, 'wb'))
random_forest_filename = '../models/random_forest_model.sav'
pickle.dump(rf, open(random_forest_filename, 'wb'))